### Setup: Import Libraries and Connect to Database

In [1]:
import sqlite3
from sqlite3 import Error
import pandas as pd

def create_connection(db_file):
    """Create a database connection to SQLite database"""
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(f"Connected to SQLite version: {sqlite3.sqlite_version}")
        return conn
    except Error as e:
        print(f"Error: {e}")
    return conn

# Connect to the database
database = "sqlite_db_pythonsqlite.db"
conn = create_connection(database)

Connected to SQLite version: 3.45.3


### Explore the Database Structure

In [2]:
# List all tables in the database
query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql_query(query, conn)
print("Tables in database:")
display(tables)

Tables in database:


,name
0,Bookings
1,Facilities
2,Members


In [3]:
# Preview Facilities table
query = "SELECT * FROM Facilities LIMIT 5;"
df_facilities = pd.read_sql_query(query, conn)
print("Facilities table preview:")
display(df_facilities)

Facilities table preview:


,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,0,Tennis Court 1,5.0,25.0,10000,200
1,1,Tennis Court 2,5.0,25.0,8000,200
2,2,Badminton Court,0.0,15.5,4000,50
3,3,Table Tennis,0.0,5.0,320,10
4,4,Massage Room 1,9.9,80.0,4000,3000


In [4]:
# Preview Members table
query = "SELECT * FROM Members LIMIT 5;"
df_members = pd.read_sql_query(query, conn)
print("Members table preview:")
display(df_members)

Members table preview:


,memid,surname,firstname,address,zipcode,telephone,recommendedby,joindate
0,0,GUEST,GUEST,GUEST,0,(000) 000-0000,,2012-07-01 00:00:00
1,1,Smith,Darren,"8 Bloomsbury Close, Boston",4321,555-555-5555,,2012-07-02 12:02:05
2,2,Smith,Tracy,"8 Bloomsbury Close, New York",4321,555-555-5555,,2012-07-02 12:08:23
3,3,Rownam,Tim,"23 Highway Way, Boston",23423,(844) 693-0723,,2012-07-03 09:32:15
4,4,Joplette,Janice,"20 Crossing Road, New York",234,(833) 942-4710,1,2012-07-03 10:25:05


In [5]:
# Preview Bookings table
query = "SELECT * FROM Bookings LIMIT 5;"
df_bookings = pd.read_sql_query(query, conn)
print("Bookings table preview:")
display(df_bookings)

Bookings table preview:


,bookid,facid,memid,starttime,slots
0,0,3,1,2012-07-03 11:00:00,2
1,1,4,1,2012-07-03 08:00:00,2
2,2,6,0,2012-07-03 18:00:00,2
3,3,7,1,2012-07-03 19:00:00,2
4,4,8,1,2012-07-03 10:00:00,1


---
## Part 1: Questions 1-9
These queries were originally run in PHPMyAdmin but can also be executed here.

---
### Q1: Facilities that Charge a Fee to Members

**Question:** Some of the facilities charge a fee to members, but some do not. Write a SQL query to produce a list of the names of the facilities that do.

In [6]:
query = """
SELECT name
FROM Facilities
WHERE membercost > 0;
"""

df_q1 = pd.read_sql_query(query, conn)
print("Q1: Facilities that charge a fee to members:")
display(df_q1)

Q1: Facilities that charge a fee to members:


,name
0,Tennis Court 1
1,Tennis Court 2
2,Massage Room 1
3,Massage Room 2
4,Squash Court


---
### Q2: Count Facilities Without Member Fees

**Question:** How many facilities do not charge a fee to members?

In [7]:
query = """
SELECT COUNT(*) AS no_fee_facilities
FROM Facilities
WHERE membercost = 0;
"""

df_q2 = pd.read_sql_query(query, conn)
print("Q2: Number of facilities that do not charge a fee:")
display(df_q2)

Q2: Number of facilities that do not charge a fee:


,no_fee_facilities
0,4


---
### Q3: Facilities with Fee < 20% of Maintenance Cost

**Question:** Write an SQL query to show a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost. Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.

In [8]:
query = """
SELECT facid, name, membercost, monthlymaintenance
FROM Facilities
WHERE membercost > 0 
  AND membercost < (monthlymaintenance * 0.2);
"""

df_q3 = pd.read_sql_query(query, conn)
print("Q3: Facilities with member fee < 20% of monthly maintenance:")
display(df_q3)

Q3: Facilities with member fee < 20% of monthly maintenance:


,facid,name,membercost,monthlymaintenance
0,0,Tennis Court 1,5.0,200
1,1,Tennis Court 2,5.0,200
2,4,Massage Room 1,9.9,3000
3,5,Massage Room 2,9.9,3000
4,6,Squash Court,3.5,80


---
### Q4: Retrieve Facilities with ID 1 and 5

**Question:** Write an SQL query to retrieve the details of facilities with ID 1 and 5. Try writing the query without using the OR operator.

In [9]:
query = """
SELECT *
FROM Facilities
WHERE facid IN (1, 5);
"""

df_q4 = pd.read_sql_query(query, conn)
print("Q4: Details of facilities with ID 1 and 5:")
display(df_q4)

Q4: Details of facilities with ID 1 and 5:


,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,1,Tennis Court 2,5.0,25,8000,200
1,5,Massage Room 2,9.9,80,4000,3000


---
### Q5: Label Facilities as Cheap or Expensive

**Question:** Produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than $100. Return the name and monthly maintenance of the facilities in question.

In [10]:
query = """
SELECT name, 
       monthlymaintenance,
       CASE WHEN monthlymaintenance > 100 THEN 'expensive'
            ELSE 'cheap' 
       END AS cost_label
FROM Facilities;
"""

df_q5 = pd.read_sql_query(query, conn)
print("Q5: Facilities labeled by maintenance cost:")
display(df_q5)

Q5: Facilities labeled by maintenance cost:


,name,monthlymaintenance,cost_label
0,Tennis Court 1,200,expensive
1,Tennis Court 2,200,expensive
2,Badminton Court,50,cheap
3,Table Tennis,10,cheap
4,Massage Room 1,3000,expensive
5,Massage Room 2,3000,expensive
6,Squash Court,80,cheap
7,Snooker Table,15,cheap
8,Pool Table,15,cheap


---
### Q6: Last Member(s) to Sign Up

**Question:** You'd like to get the first and last name of the last member(s) who signed up. Try not to use the LIMIT clause for your solution.

In [11]:
query = """
SELECT firstname, surname
FROM Members
WHERE joindate = (SELECT MAX(joindate) FROM Members);
"""

df_q6 = pd.read_sql_query(query, conn)
print("Q6: Last member(s) to sign up:")
display(df_q6)

Q6: Last member(s) to sign up:


,firstname,surname
0,Darren,Smith


---
### Q7: Members Who Used Tennis Courts

**Question:** Produce a list of all members who have used a tennis court. Include in your output the name of the court, and the name of the member formatted as a single column. Ensure no duplicate data, and order by the member name.

In [12]:
query = """
SELECT DISTINCT f.name AS court_name,
       m.firstname || ' ' || m.surname AS member_name
FROM Bookings AS b
JOIN Facilities AS f ON b.facid = f.facid
JOIN Members AS m ON b.memid = m.memid
WHERE f.name LIKE 'Tennis Court%'
ORDER BY member_name;
"""

df_q7 = pd.read_sql_query(query, conn)
print("Q7: Members who used tennis courts:")
display(df_q7)

Q7: Members who used tennis courts:


,court_name,member_name
0,Tennis Court 1,Anne Baker
1,Tennis Court 2,Anne Baker
2,Tennis Court 2,Burton Tracy
3,Tennis Court 1,Burton Tracy
4,Tennis Court 1,Charles Owen
5,Tennis Court 2,Charles Owen
6,Tennis Court 2,Darren Smith
7,Tennis Court 1,David Farrell
8,Tennis Court 2,David Farrell
9,Tennis Court 2,David Jones


---
### Q8: Bookings on 2012-09-14 Costing More Than $30 (No Subqueries)

**Question:** Produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30. Remember that guests have different costs to members (the listed costs are per half-hour 'slot'), and the guest user's ID is always 0. Include in your output the name of the facility, the name of the member formatted as a single column, and the cost. Order by descending cost, and do not use any subqueries.

In [13]:
query = """
SELECT f.name AS facility_name,
       CASE WHEN b.memid = 0 THEN 'GUEST'
            ELSE m.firstname || ' ' || m.surname 
       END AS member_name,
       CASE WHEN b.memid = 0 THEN f.guestcost * b.slots
            ELSE f.membercost * b.slots
       END AS cost
FROM Bookings AS b
JOIN Facilities AS f ON b.facid = f.facid
JOIN Members AS m ON b.memid = m.memid
WHERE DATE(b.starttime) = '2012-09-14'
  AND ((b.memid = 0 AND f.guestcost * b.slots > 30) OR
       (b.memid != 0 AND f.membercost * b.slots > 30))
ORDER BY cost DESC;
"""

df_q8 = pd.read_sql_query(query, conn)
print("Q8: Bookings on 2012-09-14 costing > $30:")
display(df_q8)

Q8: Bookings on 2012-09-14 costing > $30:


,facility_name,member_name,cost
0,Massage Room 2,GUEST,320.0
1,Massage Room 1,GUEST,160.0
2,Massage Room 1,GUEST,160.0
3,Massage Room 1,GUEST,160.0
4,Tennis Court 2,GUEST,150.0
5,Tennis Court 1,GUEST,75.0
6,Tennis Court 1,GUEST,75.0
7,Tennis Court 2,GUEST,75.0
8,Squash Court,GUEST,70.0
9,Massage Room 1,Jemima Farrell,39.6


---
### Q9: Same as Q8 But Using a Subquery

**Question:** This time, produce the same result as in Q8, but using a subquery.

In [14]:
query = """
SELECT facility_name, member_name, cost
FROM (
    SELECT f.name AS facility_name,
           CASE WHEN b.memid = 0 THEN 'GUEST'
                ELSE m.firstname || ' ' || m.surname 
           END AS member_name,
           CASE WHEN b.memid = 0 THEN f.guestcost * b.slots
                ELSE f.membercost * b.slots
           END AS cost
    FROM Bookings AS b
    JOIN Facilities AS f ON b.facid = f.facid
    JOIN Members AS m ON b.memid = m.memid
    WHERE DATE(b.starttime) = '2012-09-14'
) AS booking_costs
WHERE cost > 30
ORDER BY cost DESC;
"""

df_q9 = pd.read_sql_query(query, conn)
print("Q9: Same as Q8 but with subquery:")
display(df_q9)

Q9: Same as Q8 but with subquery:


,facility_name,member_name,cost
0,Massage Room 2,GUEST,320.0
1,Massage Room 1,GUEST,160.0
2,Massage Room 1,GUEST,160.0
3,Massage Room 1,GUEST,160.0
4,Tennis Court 2,GUEST,150.0
5,Tennis Court 1,GUEST,75.0
6,Tennis Court 1,GUEST,75.0
7,Tennis Court 2,GUEST,75.0
8,Squash Court,GUEST,70.0
9,Massage Room 1,Jemima Farrell,39.6


---
## Part 2: SQLite Questions

### Q10: Facilities with Total Revenue < 1000

**Question:** Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue. Remember that there's a different cost for guests and members!

In [15]:
query = """
SELECT f.name AS facility_name,
       SUM(CASE WHEN b.memid = 0 THEN f.guestcost * b.slots
                ELSE f.membercost * b.slots
           END) AS total_revenue
FROM Bookings AS b
JOIN Facilities AS f ON b.facid = f.facid
GROUP BY f.name
HAVING total_revenue < 1000
ORDER BY total_revenue;
"""

df_q10 = pd.read_sql_query(query, conn)
print("Q10: Facilities with revenue < $1000:")
display(df_q10)

Q10: Facilities with revenue < $1000:


,facility_name,total_revenue
0,Table Tennis,180
1,Snooker Table,240
2,Pool Table,270


---
### Q11: Members and Their Recommenders

**Question:** Produce a report of members and who recommended them in alphabetic surname, firstname order.

In [16]:
query = """
SELECT m.surname || ', ' || m.firstname AS member,
       CASE WHEN m.recommendedby IS NULL THEN 'None'
            ELSE r.surname || ', ' || r.firstname
       END AS recommended_by
FROM Members AS m
LEFT JOIN Members AS r ON m.recommendedby = r.memid
WHERE m.memid != 0
ORDER BY m.surname, m.firstname;
"""

df_q11 = pd.read_sql_query(query, conn)
print("Q11: Members and their recommenders:")
display(df_q11)

Q11: Members and their recommenders:


,member,recommended_by
0,"Bader, Florence","Stibbons, Ponder"
1,"Baker, Anne","Stibbons, Ponder"
2,"Baker, Timothy","Farrell, Jemima"
3,"Boothe, Tim","Rownam, Tim"
4,"Butters, Gerald","Smith, Darren"
5,"Coplin, Joan","Baker, Timothy"
6,"Crumpet, Erica","Smith, Tracy"
7,"Dare, Nancy","Joplette, Janice"
8,"Farrell, David",None
9,"Farrell, Jemima",None


---
### Q12: Facility Usage by Members (Not Guests)

**Question:** Find the facilities with their usage by member, but not guests.

In [17]:
query = """
SELECT f.name AS facility_name,
       m.firstname || ' ' || m.surname AS member_name,
       COUNT(*) AS booking_count
FROM Bookings AS b
JOIN Facilities AS f ON b.facid = f.facid
JOIN Members AS m ON b.memid = m.memid
WHERE b.memid != 0
GROUP BY f.name, member_name
ORDER BY f.name, booking_count DESC;
"""

df_q12 = pd.read_sql_query(query, conn)
print("Q12: Facility usage by members (first 20 rows):")
display(df_q12.head(20))

Q12: Facility usage by members (first 20 rows):


,facility_name,member_name,booking_count
0,Badminton Court,Darren Smith,132
1,Badminton Court,Tracy Smith,32
2,Badminton Court,Anna Mackenzie,30
3,Badminton Court,Gerald Butters,20
4,Badminton Court,Ponder Stibbons,16
5,Badminton Court,Tim Boothe,12
6,Badminton Court,Jack Smith,12
7,Badminton Court,Nancy Dare,10
8,Badminton Court,Anne Baker,10
9,Badminton Court,Florence Bader,9


---
### Q13: Facility Usage by Month (Not Guests)

**Question:** Find the facilities usage by month, but not guests.

In [18]:
query = """
SELECT f.name AS facility_name,
       strftime('%Y-%m', b.starttime) AS month,
       COUNT(*) AS booking_count,
       SUM(b.slots) AS total_slots
FROM Bookings AS b
JOIN Facilities AS f ON b.facid = f.facid
WHERE b.memid != 0
GROUP BY f.name, month
ORDER BY f.name, month;
"""

df_q13 = pd.read_sql_query(query, conn)
print("Q13: Facility usage by month (members only):")
display(df_q13)

Q13: Facility usage by month (members only):


,facility_name,month,booking_count,total_slots
0,Badminton Court,2012-07,51,165
1,Badminton Court,2012-08,132,414
2,Badminton Court,2012-09,161,507
3,Massage Room 1,2012-07,77,166
4,Massage Room 1,2012-08,153,316
5,Massage Room 1,2012-09,191,402
6,Massage Room 2,2012-07,4,8
7,Massage Room 2,2012-08,9,18
8,Massage Room 2,2012-09,14,28
9,Pool Table,2012-07,103,110


---
### Close Database Connection

In [19]:
# Close the connection when done
if conn:
    conn.close()
    print("Database connection closed.")

Database connection closed.
